In [136]:
class Game:

    def __init__(self):
        self.position = None
        self.event_handler = lambda *args: print(*args)

    def new_game(self):
        self.position = 0
        self.event_handler('new_game', self.position)

    def move(self, target):
        if self.position != target:
            self.position = target
            self.event_handler('move_stone', target)

    def __repr__(self):
        return str(self.position)

In [137]:
game = Game()
game

None

In [138]:
game.new_game()
game

new_game 0


0

In [139]:
game.move(1)
game

move_stone 1


1

In [140]:
import widgets_helpers


class View:
    positions = [(20, 40), (50, 50), (80, 50)]
    pt_size = 2
    stone_size = 10

    def __init__(self, game):
        game.event_handler = self.callback

        self.mcanvas = widgets_helpers.new_mcanvas(2, 100, 100)
        self.bg, self.fg = self.mcanvas
        self.fg.fill_style = 'blue'
        for x, y in self.positions:
            self.bg.fill_circle(x, y, self.pt_size)

    def place_stone(self, idx):
        self.fg.clear()
        x, y = self.positions[idx]
        self.fg.fill_circle(x, y, self.stone_size)

    def callback(self, event, idx):
        self.place_stone(idx)
        if event == 'new_game':
            self.fg.fill_text('New Game', 20, 20)

    def distance(self, src, target):
        return (src[0]-target[0])**2 + (src[1]-target[1])**2

    def get_closest(self, pos, err=20):
        dists = [(self.distance(pos, p), i) for i, p in enumerate(self.positions)]
        dist, idx = min(dists)
        if dist < err**2:
            return idx

    def _ipython_display_(self):
        display(self.mcanvas)

In [141]:
view = View(game)
view

MultiCanvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

In [142]:
view.place_stone(2)

In [143]:
view.place_stone(1)

In [144]:
pos = (85, 45)
view.get_closest(pos)

2

In [145]:
game.new_game()

In [146]:
game.move(2)

In [147]:
from canvas_callbacks import remove_all_callbacks


class Controller:
    err_out = widgets_helpers.new_output()

    def __init__(self, game, view):
        self.game = game
        self.view = view
        self.canvas = view.mcanvas

        remove_all_callbacks(self.view.mcanvas)
        self.canvas.on_key_down(self.on_key_down)
        self.canvas.on_mouse_down(self.on_mouse_down)

    @err_out.capture()
    def on_key_down(self, key, *flags):
        if key == 'n':
            self.game.new_game()

    @err_out.capture()
    def on_mouse_down(self, x, y):
        idx = self.view.get_closest((x, y))
        if idx is not None and idx != self.game.position:
            self.game.move(idx)

    def _ipython_display_(self):
        display(self.view, self.err_out)

In [148]:
controller = Controller(game, view)
controller

MultiCanvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…